In [18]:
import requests
import pandas as pd
import time
import random
import hashlib
import hmac
import base64
import json                            #解析获取到的json数据
from urllib.parse import urlencode     #使用 urllib.parse 模块来进行 URL 编码


#https://data.sd.gov.cn/portal/api/cdfecfe358d9441583cbf292d6eda4ae/detail   济宁市出租车GPS数据分页查询服务

# 请替换以下信息为你自己的实际数据
client_id = "xxxx"  #这里需要自己申请令牌id 和 密码。
token_secret = "xxxx"
api_url = "http://data.sd.gov.cn/gateway/api/1/jnczc0711"


# 要下载的总页数
total_pages = 10  # 举例，要下载10页数据
for page_num in range(1, total_pages + 1):

    #根据要求设置请求头部信息，并传到headers
    timestamp = int(time.time() * 1000)                                                       # 获取当前时间戳（毫秒）
    nonce = random.randint(0, 999999)                                                         # 生成随机数作为 nonce
    signature_string = f"{client_id}{timestamp}{nonce}"                                        # 构建待签名字符串
    signature = hmac.new(token_secret.encode(), signature_string.encode(), hashlib.sha256)    # 使用 HmacSHA256 计算签名
    signature_base64 = base64.b64encode(signature.digest()).decode()

    headers = {
        "X-Client-Id": client_id,
        "X-Timestamp": str(timestamp),
        "X-Nonce": str(nonce),
        "X-Signature": signature_base64}
    

    # 构建请求参数，并进行 URL 编码
    params = {"pagenum": page_num}
    encoded_params = urlencode(params)
    full_url = f"{api_url}?{encoded_params}"    # 构建完整的请求 URL！！！   注意官网声明的编码方式
    
    # 发起 GET 请求
    response = requests.get(full_url, headers=headers)



    if response.status_code == 200:
            data = response.json()          # 解析响应数据为 JSON 格式
            data_list = data['data']        # 获取数据列表
            df = pd.DataFrame(data_list)    # 使用pandas创建DataFrame


            
            # 保存DataFrame为csv文件
            excel_filename = r'D:\desk\vscode\济宁出租车\data_{}.csv'.format(page_num)   #每一页保存一次(每页100个)，记得修改保存路径
            df.to_csv(excel_filename, index=False,encoding="gbk")

    else:
        print("数据获取错误，请检查代码或者网络")


In [22]:
import pandas as pd
df = pd.read_csv(r"D:\desk\vscode\济宁出租车\data_5.csv",encoding='gbk')
df

,i_id,sbbh,yyzt,fx,gs,yyjssj,jd,cx,yykssj,wd,sd,cph,GPStime
0,401,64625128686,空车,146,交运,1593561600000,117,新捷达,1072915200000,35,0,鲁HT**62,1654041645000
1,402,64625129091,空车,54,北城,1655596800000,117,伊兰特,1466380800000,35,0,鲁HT**56,1654041645000
2,403,64625128802,载客,64,众鑫,1479945600000,117,新捷达,1479945600000,35,0,鲁HT**58,1654041645000
3,404,64625129077,载客,360,众鑫,1479945600000,117,吉利帝豪,1479945600000,35,21,HDQ**09,1654041645000
4,405,64625128027,载客,179,联运,1599004800000,117,新捷达,1078272000000,35,38,鲁HT**90,1654041645000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,496,64625129248,空车,90,交运,1477612800000,117,伊兰特,1477612800000,35,0,鲁HT**75,1654041655000
96,497,64625128998,空车,268,众鑫,1479427200000,117,新捷达,1479427200000,35,20,鲁HT**95,1654041655000
97,498,64625129019,载客,89,新交运,1479945600000,117,新桑塔纳,1479945600000,35,11,鲁HT**57,1654041655000
98,499,64625127861,空车,86,交运,1602892800000,116,新捷达,1064966400000,35,0,鲁HT**85,1654041655000
